# GrimAge2PAI1

## Index
1. [Instantiate model class](#Instantiate-model-class)
2. [Define clock metadata](#Define-clock-metadata)
3. [Download clock dependencies](#Download-clock-dependencies)
5. [Load features](#Load-features)
6. [Load weights into base model](#Load-weights-into-base-model)
7. [Load reference values](#Load-reference-values)
8. [Load preprocess and postprocess objects](#Load-preprocess-and-postprocess-objects)
10. [Check all clock parameters](#Check-all-clock-parameters)
10. [Basic test](#Basic-test)
11. [Save torch model](#Save-torch-model)
12. [Clear directory](#Clear-directory)

Let's first import some packages:

In [10]:
import os
import inspect
import shutil
import json
import torch
import pandas as pd
import pyaging as pya
import numpy as np

## Instantiate model class

In [11]:
def print_entire_class(cls):
    source = inspect.getsource(cls)
    print(source)

print_entire_class(pya.models.GrimAge2PAI1)

class GrimAge2PAI1(pyagingModel):
    def __init__(self):
        super().__init__()

    def preprocess(self, x):
        return x

    def postprocess(self, x):
        return x



In [12]:
model = pya.models.GrimAge2PAI1()

## Define clock metadata

In [13]:
model.metadata["clock_name"] = 'grimage2pai1'
model.metadata["data_type"] = 'methylation'
model.metadata["species"] = 'Homo sapiens'
model.metadata["year"] = 2022
model.metadata["approved_by_author"] = '⌛'
model.metadata["citation"] = "Lu, Ake T., et al. \"DNA methylation GrimAge version 2.\" Aging (Albany NY) 14.23 (2022): 9484."
model.metadata["doi"] = "https://doi.org/10.18632/aging.204434"
model.metadata["research_only"] = True
model.metadata["notes"] = None

## Download clock dependencies

In [14]:
logger = pya.logger.Logger()
urls = [
    "https://pyaging.s3.amazonaws.com/supporting_files/grimage2_subcomponents.csv",
    "https://pyaging.s3.amazonaws.com/supporting_files/grimage2.csv",
    "https://pyaging.s3.amazonaws.com/supporting_files/datMiniAnnotation3_Gold.csv",
]
dir = "."
for url in urls:
    pya.utils.download(url, dir, logger, indent_level=1)

|-----------> Data found in ./grimage2_subcomponents.csv
|-----------> Data found in ./grimage2.csv
|-----------> Data found in ./datMiniAnnotation3_Gold.csv


## Load features

#### From CSV

In [15]:
df = pd.read_csv('grimage2_subcomponents.csv', index_col=0)
df['Y.pred'].unique()

array(['DNAmGDF_15', 'DNAmB2M', 'DNAmCystatin_C', 'DNAmTIMP_1', 'DNAmadm',
       'DNAmpai_1', 'DNAmleptin', 'DNAmPACKYRS', 'DNAmlog.CRP',
       'DNAmlog.A1C'], dtype=object)

In [16]:
df = df[df['Y.pred'] == 'DNAmpai_1']
df['feature'] = df['var']
df['coefficient'] = df['beta']
model.features = df['feature'][1:].tolist()

In [17]:
df.head()

,Y.pred,var,beta,feature,coefficient
1222,DNAmpai_1,Intercept,-1129.601744,Intercept,-1129.601744
1223,DNAmpai_1,cg10143960,62.578405,cg10143960,62.578405
1224,DNAmpai_1,cg07284273,321.195548,cg07284273,321.195548
1225,DNAmpai_1,cg17747265,-476.725772,cg17747265,-476.725772
1226,DNAmpai_1,cg09971184,6221.585387,cg09971184,6221.585387


## Load weights into base model

#### Linear model

In [18]:
weights = torch.tensor(df['coefficient'][1:].tolist()).unsqueeze(0)
intercept = torch.tensor([df['coefficient'].iloc[0]])

#### Linear model

In [19]:
base_model = pya.models.LinearModel(input_dim=len(model.features))

base_model.linear.weight.data = weights.float()
base_model.linear.bias.data = intercept.float()

model.base_model = base_model

## Load reference values

In [20]:
reference_df = pd.read_csv('datMiniAnnotation3_Gold.csv', index_col=0)
model.reference_values = reference_df.loc[model.features[0:]]['gold'].tolist()

## Load preprocess and postprocess objects

In [21]:
model.preprocess_name = None
model.preprocess_dependencies = None

In [22]:
model.postprocess_name = None
model.postprocess_dependencies = None

## Check all clock parameters

In [23]:
pya.utils.print_model_details(model)


%==================================== Model Details ====================================%
Model Attributes:

training: True
metadata: {'approved_by_author': '⌛',
 'citation': 'Lu, Ake T., et al. "DNA methylation GrimAge version 2." Aging '
             '(Albany NY) 14.23 (2022): 9484.',
 'clock_name': 'grimage2pai1',
 'data_type': 'methylation',
 'doi': 'https://doi.org/10.18632/aging.204434',
 'notes': None,
 'research_only': True,
 'species': 'Homo sapiens',
 'version': None,
 'year': 2022}
reference_values: [0.259761798520256, 0.111037942492215, 0.29422000126188, 0.936304436158017, 0.848665856510958, 0.644492617929823, 0.893780204840225, 0.0721244934513398, 0.884599101342416, 0.930445913821681, 0.655185531462437, 0.902782438241089, 0.183574093402218, 0.072529951052299, 0.0175800874436762, 0.0281519446957868, 0.583273419793119, 0.744700209451187, 0.0410865998356346, 0.94438869621023, 0.9447673872075, 0.107357432919823, 0.02228793678189, 0.95796467775612, 0.330065094457947, 0.8354705

## Basic test

In [24]:
torch.manual_seed(42)
input = torch.randn(10, len(model.features), dtype=float).double()
model.eval()
model.to(float)
pred = model(input)
pred

tensor([[-167391.7937],
        [  35688.6704],
        [ 339492.3748],
        [ 311586.7829],
        [ -23585.9907],
        [  25749.6604],
        [ 214765.9318],
        [  46047.1643],
        [ 276867.1905],
        [ -47640.9441]], dtype=torch.float64, grad_fn=<AddmmBackward0>)

## Save torch model

In [25]:
torch.save(model, f"../weights/{model.metadata['clock_name']}.pt")

## Clear directory
<a id="10"></a>

In [26]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)

Deleted file: grimage2_subcomponents.csv
Deleted file: datMiniAnnotation3_Gold.csv
Deleted file: grimage2.csv
